In [ ]:
## 구글 스프레드시트와 연계

In [ ]:
3


: 

In [2]:
import random
from itertools import combinations

def generate_matches(male_players, female_players):
    mixed_teams = list(combinations(male_players, 2)) + list(combinations(female_players, 2))
    random.shuffle(mixed_teams)
    return mixed_teams

def organize_tournament(male_count, female_count, num_courts):
    male_players = list(range(1, male_count + 1))
    female_players = list(range(1, female_count + 1))
    
    # Shuffle player lists to randomize pairing
    random.shuffle(male_players)
    random.shuffle(female_players)
    
    # Prepare courts
    courts = [[] for _ in range(num_courts)]
    
    # Assign mixed doubles, male doubles, female doubles
    for i in range(num_courts):
        mixed_matches = generate_matches(male_players, female_players)
        courts[i].extend(mixed_matches)
    
    return courts

# Given data
num_males = 50
num_females = 34
num_courts = 6

# Organize the tournament
tournament_schedule = organize_tournament(num_males, num_females, num_courts)

# Print the tournament schedule
for i, court in enumerate(tournament_schedule):
    print(f"Court {i+1}:")
    for match in court:
        print(f"  {match}")



3

## 건대 정기코트 운영을 위한 자동 대진 코드 작성
요구사항 
 - 1코트 랜덤매칭생성
 - 기본적으로, 경기 수를 맞추는 방식
 - 여자 4명이 되면, 여복을 진행하고 그게 아니면 혼복을 진행



In [29]:
## 2024 09 29

import random
from collections import defaultdict

def generate_fair_matchups(participants, total_matches, players_per_match=4):
    random.shuffle(participants)  # 참가자 목록을 무작위로 섞음
    matchups = []
    
    # 각 참가자가 만난 적이 있는 사람을 기록할 딕셔너리
    match_history = defaultdict(lambda: defaultdict(int))
    
    # 각 참가자의 경기 출전 횟수를 기록할 딕셔너리
    match_counts = defaultdict(int)
    
    # 초기화: 각 참가자의 자기 자신과의 매칭 기록을 0으로 설정
    for player in participants:
        match_history[player][player] = 0
    
    for _ in range(total_matches):
        match = []
        available_players = participants[:]  # 참가자 전체 리스트를 복사
        
        # 매치에 들어갈 4명을 선정
        while len(match) < players_per_match and available_players:
            # 현재 매치에 있는 사람들과 가능한 한 만난 적이 없는 사람을 선택
            available_players.sort(key=lambda p: sum(match_history[p][opponent] for opponent in match))
            selected_player = available_players.pop(0)
            match.append(selected_player)
        
        # 매치가 완성되면 저장
        matchups.append(sorted(match))  # 매치에 참가한 사람을 정렬
        
        # 매치 기록 갱신: 매치에 참가한 사람들이 서로 만난 횟수를 기록
        for player in match:
            match_counts[player] += 1  # 출전 횟수 증가
            for opponent in match:
                if player != opponent:
                    match_history[player][opponent] += 1
        
        # 무작위로 섞어 다음 경기에도 다양성을 높임
        random.shuffle(participants)
    
    return matchups, match_history, match_counts

# 예시 참가자 리스트
participants = ["성현", "정현", "예스욱",
                # "쥬드", "챨리심", "태보", "Kohei",
                "파란하늘", "유하"
                # "미지", "김연주"
                ###################
                , "태규"  # 게스트
                ##################
                ]

# 원하는 총 경기 수 (예: 5경기)
total_matches = 6

# 매칭 결과 생성
matchups, match_history, match_counts = generate_fair_matchups(participants, total_matches)

# 매칭 결과 출력 (스프레드시트에 붙여넣기 용이하게)
print("\nMatchups:")
for idx, match in enumerate(matchups):
    print(f"Match {idx + 1}: " + "\t".join(match))

# 각 참가자의 만난 횟수 출력 (dictionary 형태로 정렬된 상태)
print("\nPlayer match history:")
for player in sorted(match_history.keys()):
    sorted_opponents = {opponent: match_history[player][opponent] for opponent in sorted(match_history[player].keys())}
    print(f"{player}: {sorted_opponents}")

# 각 참가자의 경기 출전 횟수 출력
print("\nPlayer match counts:")
for player in sorted(match_counts.keys()):
    print(f"{player}: {match_counts[player]} matches")



Matchups:
Match 1: 성현	예스욱	유하	태규
Match 2: 성현	유하	정현	파란하늘
Match 3: 성현	예스욱	정현	태규
Match 4: 예스욱	유하	정현	파란하늘
Match 5: 유하	정현	태규	파란하늘
Match 6: 성현	예스욱	태규	파란하늘

Player match history:
성현: {'성현': 0, '예스욱': 3, '유하': 2, '정현': 2, '태규': 3, '파란하늘': 2}
예스욱: {'성현': 3, '예스욱': 0, '유하': 2, '정현': 2, '태규': 3, '파란하늘': 2}
유하: {'성현': 2, '예스욱': 2, '유하': 0, '정현': 3, '태규': 2, '파란하늘': 3}
정현: {'성현': 2, '예스욱': 2, '유하': 3, '정현': 0, '태규': 2, '파란하늘': 3}
태규: {'성현': 3, '예스욱': 3, '유하': 2, '정현': 2, '태규': 0, '파란하늘': 2}
파란하늘: {'성현': 2, '예스욱': 2, '유하': 3, '정현': 3, '태규': 2, '파란하늘': 0}

Player match counts:
성현: 4 matches
예스욱: 4 matches
유하: 4 matches
정현: 4 matches
태규: 4 matches
파란하늘: 4 matches


In [32]:
## ####기본 매칭

import random
from collections import defaultdict

def generate_fair_matchups(participants, total_matches, players_per_match=4):
    random.shuffle(participants)  # 참가자 목록을 무작위로 섞음
    matchups = []
    
    # 각 참가자가 만난 적이 있는 사람을 기록할 딕셔너리
    match_history = defaultdict(lambda: defaultdict(int))
    
    # 각 참가자의 경기 출전 횟수를 기록할 딕셔너리
    match_counts = defaultdict(int)
    
    # 초기화: 각 참가자의 자기 자신과의 매칭 기록을 0으로 설정
    for player in participants:
        match_history[player][player] = 0
    
    for _ in range(total_matches):
        match = []
        available_players = participants[:]  # 참가자 전체 리스트를 복사
        
        # 매치에 들어갈 4명을 선정
        while len(match) < players_per_match and available_players:
            # 현재 매치에 있는 사람들과 가능한 한 만난 적이 없는 사람을 선택
            available_players.sort(key=lambda p: sum(match_history[p][opponent] for opponent in match))
            selected_player = available_players.pop(0)
            match.append(selected_player)
        
        # 매치가 완성되면 저장
        matchups.append(sorted(match))  # 매치에 참가한 사람을 정렬
        
        # 매치 기록 갱신: 매치에 참가한 사람들이 서로 만난 횟수를 기록
        for player in match:
            match_counts[player] += 1  # 출전 횟수 증가
            for opponent in match:
                if player != opponent:
                    match_history[player][opponent] += 1
        
        # 무작위로 섞어 다음 경기에도 다양성을 높임
        random.shuffle(participants)
    
    return matchups, match_history, match_counts

# 예시 참가자 리스트
participants = ["성현", "정현", "쥬드", "챨리심", "태보", 
                "쿤", "예스욱", "김연주", 
                # "미지", 
                "유하"]

# 원하는 총 경기 수 (예: 5경기)
total_matches = 9

# 매칭 결과 생성
matchups, match_history, match_counts = generate_fair_matchups(participants, total_matches)

# 매칭 결과 출력 (스프레드시트에 붙여넣기 용이하게)
print("\nMatchups:")
for idx, match in enumerate(matchups):
    print(f"Match {idx + 1}: " + "\t".join(match))

# 각 참가자의 만난 횟수 출력 (dictionary 형태로 정렬된 상태)
print("\nPlayer match history:")
for player in sorted(match_history.keys()):
    sorted_opponents = {opponent: match_history[player][opponent] for opponent in sorted(match_history[player].keys())}
    print(f"{player}: {sorted_opponents}")

# 각 참가자의 경기 출전 횟수 출력
print("\nPlayer match counts:")
for player in sorted(match_counts.keys()):
    print(f"{player}: {match_counts[player]} matches")



Matchups:
Match 1: 유하	정현	쥬드	태보
Match 2: 성현	예스욱	정현	챨리심
Match 3: 김연주	성현	유하	쿤
Match 4: 김연주	예스욱	챨리심	태보
Match 5: 예스욱	유하	쥬드	쿤
Match 6: 김연주	유하	정현	챨리심
Match 7: 김연주	성현	쥬드	태보
Match 8: 쥬드	챨리심	쿤	태보
Match 9: 예스욱	정현	쿤	태보

Player match history:
김연주: {'김연주': 0, '성현': 2, '예스욱': 1, '유하': 2, '정현': 1, '쥬드': 1, '챨리심': 2, '쿤': 1, '태보': 2}
성현: {'김연주': 2, '성현': 0, '예스욱': 1, '유하': 1, '정현': 1, '쥬드': 1, '챨리심': 1, '쿤': 1, '태보': 1}
예스욱: {'김연주': 1, '성현': 1, '예스욱': 0, '유하': 1, '정현': 2, '쥬드': 1, '챨리심': 2, '쿤': 2, '태보': 2}
유하: {'김연주': 2, '성현': 1, '예스욱': 1, '유하': 0, '정현': 2, '쥬드': 2, '챨리심': 1, '쿤': 2, '태보': 1}
정현: {'김연주': 1, '성현': 1, '예스욱': 2, '유하': 2, '정현': 0, '쥬드': 1, '챨리심': 2, '쿤': 1, '태보': 2}
쥬드: {'김연주': 1, '성현': 1, '예스욱': 1, '유하': 2, '정현': 1, '쥬드': 0, '챨리심': 1, '쿤': 2, '태보': 3}
챨리심: {'김연주': 2, '성현': 1, '예스욱': 2, '유하': 1, '정현': 2, '쥬드': 1, '챨리심': 0, '쿤': 1, '태보': 2}
쿤: {'김연주': 1, '성현': 1, '예스욱': 2, '유하': 2, '정현': 1, '쥬드': 2, '챨리심': 1, '쿤': 0, '태보': 2}
태보: {'김연주': 2, '성현': 1, '예스욱': 2, '유하': 1, '정현': 2, '쥬드': 3, '챨

In [35]:
import random
from collections import defaultdict

def generate_fair_matchups(participants, total_matches, players_per_match=4):
    random.shuffle(participants)  # 참가자 목록을 무작위로 섞음
    matchups = []
    
    # 각 참가자가 만난 적이 있는 사람을 기록할 딕셔너리
    match_history = defaultdict(lambda: defaultdict(int))
    
    # 각 참가자의 경기 출전 횟수를 기록할 딕셔너리
    match_counts = defaultdict(int)
    
    # 초기화: 각 참가자의 자기 자신과의 매칭 기록을 0으로 설정
    for player in participants:
        match_history[player][player] = 0
    
    for _ in range(total_matches):
        match = []
        available_players = participants[:]  # 참가자 전체 리스트를 복사
        
        # 매치에 들어갈 4명을 선정
        while len(match) < players_per_match and available_players:
            # 현재 매치에 있는 사람들과 가능한 한 만난 적이 없는 사람을 선택
            available_players.sort(key=lambda p: (sum(match_history[p][opponent] for opponent in match), match_counts[p]))
            selected_player = available_players.pop(0)
            match.append(selected_player)
        
        # 매치가 완성되면 저장
        matchups.append(sorted(match))  # 매치에 참가한 사람을 정렬
        
        # 매치 기록 갱신: 매치에 참가한 사람들이 서로 만난 횟수를 기록
        for player in match:
            match_counts[player] += 1  # 출전 횟수 증가
            for opponent in match:
                if player != opponent:
                    match_history[player][opponent] += 1
        
        # 무작위로 섞어 다음 경기에도 다양성을 높임
        random.shuffle(participants)
    
    return matchups, match_history, match_counts

# 예시 참가자 리스트
participants = ["성현", "정현", "쥬드", "챨리심", "태보", 
                # "쿤", "예스욱", 
                "김연주", "유하"]

# 원하는 총 경기 수 (예: 9경기)
total_matches = 3

# 매칭 결과 생성
matchups, match_history, match_counts = generate_fair_matchups(participants, total_matches)

# 매칭 결과 출력 (스프레드시트에 붙여넣기 용이하게)
print("\nMatchups:")
for idx, match in enumerate(matchups):
    print(f"Match {idx + 1}: " + "\t".join(match))

# 각 참가자의 만난 횟수 출력 (dictionary 형태로 정렬된 상태)
print("\nPlayer match history:")
for player in sorted(match_history.keys()):
    sorted_opponents = {opponent: match_history[player][opponent] for opponent in sorted(match_history[player].keys())}
    print(f"{player}: {sorted_opponents}")

# 각 참가자의 경기 출전 횟수 출력
print("\nPlayer match counts:")
for player in sorted(match_counts.keys()):
    print(f"{player}: {match_counts[player]} matches")


Matchups:
Match 1: 정현	쥬드	챨리심	태보
Match 2: 김연주	성현	유하	정현
Match 3: 김연주	성현	챨리심	태보

Player match history:
김연주: {'김연주': 0, '성현': 2, '유하': 1, '정현': 1, '쥬드': 0, '챨리심': 1, '태보': 1}
성현: {'김연주': 2, '성현': 0, '유하': 1, '정현': 1, '쥬드': 0, '챨리심': 1, '태보': 1}
유하: {'김연주': 1, '성현': 1, '유하': 0, '정현': 1, '쥬드': 0, '챨리심': 0, '태보': 0}
정현: {'김연주': 1, '성현': 1, '유하': 1, '정현': 0, '쥬드': 1, '챨리심': 1, '태보': 1}
쥬드: {'김연주': 0, '성현': 0, '유하': 0, '정현': 1, '쥬드': 0, '챨리심': 1, '태보': 1}
챨리심: {'김연주': 1, '성현': 1, '유하': 0, '정현': 1, '쥬드': 1, '챨리심': 0, '태보': 2}
태보: {'김연주': 1, '성현': 1, '유하': 0, '정현': 1, '쥬드': 1, '챨리심': 2, '태보': 0}

Player match counts:
김연주: 2 matches
성현: 2 matches
유하: 1 matches
정현: 2 matches
쥬드: 1 matches
챨리심: 2 matches
태보: 2 matches


In [36]:
from collections import defaultdict

# 주어진 경기 데이터
matches = [
    ["성현", "정현", "쥬드", "태보"],
    ["김연주", "성현", "유하", "쿤"],
    ["쿤", "쥬드", "챨리심", "정현"],
    ["유하", "정현", "김연주", "태보"],
    ["쿤", "성현", "챨리심", "태보"],
    ["김연주", "쥬드", "유하", "챨리심"],
    ["성현", "예스욱", "쿤", "정현"],
    ["김연주", "정현", "유하", "성현"],
    ["예스욱", "태보", "챨리심", "쥬드"],
    ["김연주", "태보", "유하", "쥬드"],
    ["성현", "정현", "쥬드", "챨리심"],
    ["성현", "쿤", "정현", "태보"]
]

# 참가자 목록 추출
participants = set()
for match in matches:
    participants.update(match)

participants = list(participants)

# 초기화: 모든 참가자를 키로 하고 값은 0으로 설정
pairwise_counts = defaultdict(lambda: defaultdict(int))
co_players_counts = defaultdict(lambda: defaultdict(int))
match_counts = defaultdict(int)

# 각 참가자의 같은 경기 출전 기록을 업데이트
for match in matches:
    # 페어 계산 (0번1번, 2번3번)
    pairs = [(match[0], match[1]), (match[2], match[3])]
    
    for p1, p2 in pairs:
        pairwise_counts[p1][p2] += 1
        pairwise_counts[p2][p1] += 1
    
    # 같은 경기에서 출전한 사람들 간의 횟수 계산
    for i in range(len(match)):
        for j in range(i + 1, len(match)):
            player1 = match[i]
            player2 = match[j]
            co_players_counts[player1][player2] += 1
            co_players_counts[player2][player1] += 1
    
    # 각 참가자의 경기 출전 횟수를 업데이트
    for player in match:
        match_counts[player] += 1

# 페어한 사람의 딕셔너리와 같이 경기한 사람의 딕셔너리 출력
print("Pairwise match counts (sorted by count):")
sorted_pairwise_counts = {player: dict(sorted(pairwise_counts[player].items(), key=lambda item: item[1], reverse=True))
                          for player in participants}

for player in sorted(participants, key=lambda p: sum(pairwise_counts[p].values()), reverse=True):
    pair_counts = sorted_pairwise_counts[player]
    # 모든 참가자를 포함한 딕셔너리 생성
    all_pairs = {p: 0 for p in participants}
    all_pairs.update(pair_counts)
    print(f"{player}: {all_pairs}")

print("\nCo-players list with counts (sorted by count):")
for player in sorted(participants, key=lambda p: match_counts[p], reverse=True):
    co_players_with_counts = dict(sorted(co_players_counts[player].items(), key=lambda item: item[1], reverse=True))
    # 모든 참가자를 포함한 딕셔너리 생성
    all_co_players = {p: 0 for p in participants}
    all_co_players.update(co_players_with_counts)
    print(f"{player}: {all_co_players}")

# 각 참가자의 총 경기 출전 횟수 출력
print("\nTotal match counts:")
for player in sorted(participants, key=lambda p: match_counts[p], reverse=True):
    print(f"{player}: {match_counts[player]} matches")

Pairwise match counts (sorted by count):
정현: {'정현': 0, '유하': 1, '태보': 1, '쥬드': 0, '쿤': 1, '성현': 2, '챨리심': 1, '예스욱': 0, '김연주': 1}
성현: {'정현': 2, '유하': 1, '태보': 0, '쥬드': 0, '쿤': 2, '성현': 0, '챨리심': 0, '예스욱': 1, '김연주': 1}
태보: {'정현': 1, '유하': 0, '태보': 0, '쥬드': 1, '쿤': 0, '성현': 0, '챨리심': 1, '예스욱': 1, '김연주': 2}
쥬드: {'정현': 0, '유하': 1, '태보': 1, '쥬드': 0, '쿤': 1, '성현': 0, '챨리심': 2, '예스욱': 0, '김연주': 1}
유하: {'정현': 1, '유하': 0, '태보': 0, '쥬드': 1, '쿤': 1, '성현': 1, '챨리심': 1, '예스욱': 0, '김연주': 0}
쿤: {'정현': 1, '유하': 1, '태보': 0, '쥬드': 1, '쿤': 0, '성현': 2, '챨리심': 0, '예스욱': 0, '김연주': 0}
챨리심: {'정현': 1, '유하': 1, '태보': 1, '쥬드': 2, '쿤': 0, '성현': 0, '챨리심': 0, '예스욱': 0, '김연주': 0}
김연주: {'정현': 1, '유하': 0, '태보': 2, '쥬드': 1, '쿤': 0, '성현': 1, '챨리심': 0, '예스욱': 0, '김연주': 0}
예스욱: {'정현': 0, '유하': 0, '태보': 1, '쥬드': 0, '쿤': 0, '성현': 1, '챨리심': 0, '예스욱': 0, '김연주': 0}

Co-players list with counts (sorted by count):
정현: {'정현': 0, '유하': 2, '태보': 3, '쥬드': 3, '쿤': 3, '성현': 5, '챨리심': 2, '예스욱': 1, '김연주': 2}
성현: {'정현': 5, '유하': 2, '태보': 

In [6]:
from collections import defaultdict

# 주어진 경기 데이터 (업데이트된 데이터)
matches = [
    ["미지", "성현", "파란하늘", "태보"],
    ["예스욱", "정현", "쥬드", "성현"],
    ["미지", "챨리심", "파란하늘", "예스욱"],
    ["예스욱", "태보", "챨리심", "정현"],
    ["성현", "태보", "파란하늘", "쥬드"],
    ["미지", "쥬드", "챨리심", "정현"],
    ["미지", "성현", "파란하늘", "쥬드"],
    ["성현", "예스욱", "태보", "챨리심"],
    ["미지", "태보", "파란하늘", "정현"]
]

# 참가자 목록 추출
participants = set()
for match in matches:
    participants.update(match)

participants = list(participants)

# 초기화: 모든 참가자를 키로 하고 값은 0으로 설정
pairwise_counts = defaultdict(lambda: defaultdict(int))
co_players_counts = defaultdict(lambda: defaultdict(int))
match_counts = defaultdict(int)

# 각 참가자의 같은 경기 출전 기록을 업데이트
for match in matches:
    # 페어 계산 (0번1번, 2번3번)
    pairs = [(match[0], match[1]), (match[2], match[3])]
    
    for p1, p2 in pairs:
        pairwise_counts[p1][p2] += 1
        pairwise_counts[p2][p1] += 1
    
    # 같은 경기에서 출전한 사람들 간의 횟수 계산
    for i in range(len(match)):
        for j in range(i + 1, len(match)):
            player1 = match[i]
            player2 = match[j]
            co_players_counts[player1][player2] += 1
            co_players_counts[player2][player1] += 1
    
    # 각 참가자의 경기 출전 횟수를 업데이트
    for player in match:
        match_counts[player] += 1

# 페어한 사람의 딕셔너리와 같이 경기한 사람의 딕셔너리 출력
print("Pairwise match counts (sorted by count):")
sorted_pairwise_counts = {player: dict(sorted(pairwise_counts[player].items(), key=lambda item: item[1], reverse=True))
                          for player in participants}

for player in sorted(participants, key=lambda p: sum(pairwise_counts[p].values()), reverse=True):
    pair_counts = sorted_pairwise_counts[player]
    # 모든 참가자를 포함한 딕셔너리 생성
    all_pairs = {p: 0 for p in participants}
    all_pairs.update(pair_counts)
    print(f"{player}: {all_pairs}")

print("\nCo-players list with counts (sorted by count):")
for player in sorted(participants, key=lambda p: match_counts[p], reverse=True):
    co_players_with_counts = dict(sorted(co_players_counts[player].items(), key=lambda item: item[1], reverse=True))
    # 모든 참가자를 포함한 딕셔너리 생성
    all_co_players = {p: 0 for p in participants}
    all_co_players.update(co_players_with_counts)
    print(f"{player}: {all_co_players}")

Pairwise match counts (sorted by count):
미지: {'미지': 0, '태보': 1, '챨리심': 1, '쥬드': 1, '성현': 2, '예스욱': 0, '정현': 0, '파란하늘': 0}
태보: {'미지': 1, '태보': 0, '챨리심': 1, '쥬드': 0, '성현': 1, '예스욱': 1, '정현': 0, '파란하늘': 1}
성현: {'미지': 2, '태보': 1, '챨리심': 0, '쥬드': 1, '성현': 0, '예스욱': 1, '정현': 0, '파란하늘': 0}
파란하늘: {'미지': 0, '태보': 1, '챨리심': 0, '쥬드': 2, '성현': 0, '예스욱': 1, '정현': 1, '파란하늘': 0}
챨리심: {'미지': 1, '태보': 1, '챨리심': 0, '쥬드': 0, '성현': 0, '예스욱': 0, '정현': 2, '파란하늘': 0}
쥬드: {'미지': 1, '태보': 0, '챨리심': 0, '쥬드': 0, '성현': 1, '예스욱': 0, '정현': 0, '파란하늘': 2}
예스욱: {'미지': 0, '태보': 1, '챨리심': 0, '쥬드': 0, '성현': 1, '예스욱': 0, '정현': 1, '파란하늘': 1}
정현: {'미지': 0, '태보': 0, '챨리심': 2, '쥬드': 0, '성현': 0, '예스욱': 1, '정현': 0, '파란하늘': 1}

Co-players list with counts (sorted by count):
미지: {'미지': 0, '태보': 2, '챨리심': 2, '쥬드': 2, '성현': 2, '예스욱': 1, '정현': 2, '파란하늘': 4}
태보: {'미지': 2, '태보': 0, '챨리심': 2, '쥬드': 1, '성현': 3, '예스욱': 2, '정현': 2, '파란하늘': 3}
성현: {'미지': 2, '태보': 3, '챨리심': 1, '쥬드': 3, '성현': 0, '예스욱': 2, '정현': 1, '파란하늘': 3}
파란하늘: {'미지': 4, 

In [36]:
### 9월 15일 매치

## 기본 매칭

import random
from collections import defaultdict

# random.seed(20240922)

def generate_fair_matchups(participants, total_matches, players_per_match=4):
    random.shuffle(participants)  # 참가자 목록을 무작위로 섞음
    matchups = []
    
    # 각 참가자가 만난 적이 있는 사람을 기록할 딕셔너리
    match_history = defaultdict(lambda: defaultdict(int))
    
    # 각 참가자의 경기 출전 횟수를 기록할 딕셔너리
    match_counts = defaultdict(int)
    
    # 초기화: 각 참가자의 자기 자신과의 매칭 기록을 0으로 설정
    for player in participants:
        match_history[player][player] = 0
    
    for _ in range(total_matches):
        match = []
        available_players = participants[:]  # 참가자 전체 리스트를 복사
        
        # 매치에 들어갈 4명을 선정
        while len(match) < players_per_match and available_players:
            # 현재 매치에 있는 사람들과 가능한 한 만난 적이 없는 사람을 선택
            available_players.sort(key=lambda p: sum(match_history[p][opponent] for opponent in match))
            selected_player = available_players.pop(0)
            match.append(selected_player)
        
        # 매치가 완성되면 저장
        matchups.append(sorted(match))  # 매치에 참가한 사람을 정렬
        
        # 매치 기록 갱신: 매치에 참가한 사람들이 서로 만난 횟수를 기록
        for player in match:
            match_counts[player] += 1  # 출전 횟수 증가
            for opponent in match:
                if player != opponent:
                    match_history[player][opponent] += 1
        
        # 무작위로 섞어 다음 경기에도 다양성을 높임
        random.shuffle(participants)
    
    return matchups, match_history, match_counts

# 예시 참가자 리스트
participants = [
'김연주',
'미지',
'정현',
'쥬드',
'챨리심',
'태보',
'파란하늘',
'Kohei Tada',   
]

# 원하는 총 경기 수 (예: 5경기)
total_matches = 9

# 매칭 결과 생성
matchups, match_history, match_counts = generate_fair_matchups(participants, total_matches)

# 매칭 결과 출력 (스프레드시트에 붙여넣기 용이하게)
print("\nMatchups:")
for idx, match in enumerate(matchups):
    print(f"Match {idx + 1}: " + "\t".join(match))

# 각 참가자의 만난 횟수 출력 (dictionary 형태로 정렬된 상태)
print("\nPlayer match history:")
for player in sorted(match_history.keys()):
    sorted_opponents = {opponent: match_history[player][opponent] for opponent in sorted(match_history[player].keys())}
    print(f"{player}: {sorted_opponents}")

# 각 참가자의 경기 출전 횟수 출력
print("\nPlayer match counts:")
for player in sorted(match_counts.keys()):
    print(f"{player}: {match_counts[player]} matches")



Matchups:
Match 1: 미지	정현	쥬드	챨리심
Match 2: 김연주	정현	태보	파란하늘
Match 3: Kohei Tada	김연주	정현	쥬드
Match 4: 김연주	미지	챨리심	태보
Match 5: Kohei Tada	쥬드	태보	파란하늘
Match 6: 미지	쥬드	태보	파란하늘
Match 7: Kohei Tada	정현	챨리심	파란하늘
Match 8: Kohei Tada	미지	정현	태보
Match 9: Kohei Tada	김연주	쥬드	챨리심

Player match history:
Kohei Tada: {'Kohei Tada': 0, '김연주': 2, '미지': 1, '정현': 3, '쥬드': 3, '챨리심': 2, '태보': 2, '파란하늘': 2}
김연주: {'Kohei Tada': 2, '김연주': 0, '미지': 1, '정현': 2, '쥬드': 2, '챨리심': 2, '태보': 2, '파란하늘': 1}
미지: {'Kohei Tada': 1, '김연주': 1, '미지': 0, '정현': 2, '쥬드': 2, '챨리심': 2, '태보': 3, '파란하늘': 1}
정현: {'Kohei Tada': 3, '김연주': 2, '미지': 2, '정현': 0, '쥬드': 2, '챨리심': 2, '태보': 2, '파란하늘': 2}
쥬드: {'Kohei Tada': 3, '김연주': 2, '미지': 2, '정현': 2, '쥬드': 0, '챨리심': 2, '태보': 2, '파란하늘': 2}
챨리심: {'Kohei Tada': 2, '김연주': 2, '미지': 2, '정현': 2, '쥬드': 2, '챨리심': 0, '태보': 1, '파란하늘': 1}
태보: {'Kohei Tada': 2, '김연주': 2, '미지': 3, '정현': 2, '쥬드': 2, '챨리심': 1, '태보': 0, '파란하늘': 3}
파란하늘: {'Kohei Tada': 2, '김연주': 1, '미지': 1, '정현': 2, '쥬드': 2, '챨리심': 1, '태보': 3, '파란하늘': 